In [1]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `AWS_ACCESS_KEY_ID`, `AWS_BUCKET`, `AWS_SECRET_ACCESS_KEY`
* `XCUBE_GEN_API_SERVER_PORT`, `XCUBE_GEN_API_SERVER_URL`, `XCUBE_GEN_AUTH_AUD`, `XCUBE_GEN_AUTH_CLIENT_ID`, `XCUBE_GEN_AUTH_CLIENT_SECRET`, `XCUBE_GEN_AUTH_DOMAIN`, `XCUBE_GEN_USER`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DAPA_URL`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


In [2]:
import os
os.environ["XCUBE_GEN_API_USER_NAME"] = os.environ["XCUBE_GEN_USER"]

In [3]:
from edc import check_compatibility
check_compatibility("user-0.19.4")

This notebook is compatible with this base image version (user-0.19.4).

## xcube Generator Python Access Library

This notebook shows how to generate xcube cube using the xcube-gen web service [xcube-gen.brockmann-consult.de](https://xcube-gen.brockmann-consult.de).
Please be aware, this notebook will not run unless you have access to the xcube-gen service as well as a bucket on AWS.

In [4]:
from job_api import JobApi
api = JobApi()
api.whoami

'hub@eox.at'

## Generate a config

In [5]:
import os
cfg = {
  "input_configs": [
    {
      "store_id": "@sentinelhub",
      "data_id": "S2L2A",
      "open_params": {
        "tile_size": [
          1000,
          1000
        ]
      }
    }
  ],
  "cube_config": {
    "variable_names": [
      "B01",
      "B02"
    ],
    "bbox": [
      7,
      53,
      9,
      55
    ],
    "spatial_res": 0.001,
    "crs": "WGS84",
    "time_range": [
      "2000-06-20",
      "2000-06-22"
    ],
    "time_period": "1D"
  },
  "output_config": {
    "store_id": "s3",
    "store_params": {
      "bucket_name": os.environ["AWS_BUCKET"],
      "aws_access_key_id": os.environ["AWS_ACCESS_KEY_ID"],
      "aws_secret_access_key": os.environ["AWS_SECRET_ACCESS_KEY"],
    }
  }
}

## Generating an xcube

In [6]:
response = api.create(cfg=cfg)
job_id = response['result']['job_id']
response

{'result': {'job_id': 'xcube-gen-81cbc435-0a2e-48cf-b021-13463c737379',
  'status': {'active': None,
   'completion_time': None,
   'conditions': None,
   'failed': None,
   'start_time': None,
   'succeeded': None}},
 'status': 'ok'}

## Getting the Status of a Generation Job

In [7]:
# wait until job has been created
import time
time.sleep(3)

In [8]:
api.status(job_id)

{'result': {'job_id': 'xcube-gen-81cbc435-0a2e-48cf-b021-13463c737379',
  'output': [],
  'status': {'active': 1,
   'completion_time': None,
   'conditions': None,
   'failed': None,
   'start_time': 'Mon, 13 Jul 2020 11:34:30 GMT',
   'succeeded': None}},
 'status': 'ok'}

## Listing my Jobs

In [9]:
api.list()

{'result': [{'job_id': 'xcube-gen-2d35fa63-a104-4b14-b2a6-0dba7fa1d436',
   'status': {'active': None,
    'completion_time': 'Thu, 09 Jul 2020 09:02:51 GMT',
    'failed': None,
    'start_time': 'Thu, 09 Jul 2020 09:02:41 GMT',
    'succeeded': 1}},
  {'job_id': 'xcube-gen-38db633e-3703-433e-b3f1-8d22a0c5ac20',
   'status': {'active': None,
    'completion_time': None,
    'failed': 2,
    'start_time': 'Tue, 07 Jul 2020 12:10:32 GMT',
    'succeeded': None}},
  {'job_id': 'xcube-gen-81cbc435-0a2e-48cf-b021-13463c737379',
   'status': {'active': 1,
    'completion_time': None,
    'failed': None,
    'start_time': 'Mon, 13 Jul 2020 11:34:30 GMT',
    'succeeded': None}}],
 'status': 'ok'}

## Deleting a job

In [10]:
api.delete(job_id)

{'result': 'Success', 'status': 'ok'}